# 📊 Logging

> One-time logging configuration for all fh_saas modules.

In [ ]:
#| default_exp utils_log

In [ ]:
#| export
import os
import logging
import logging.handlers
from pathlib import Path

## 🎯 Overview

| Function | Purpose |
|----------|---------|
| `configure_logging` | One-time setup at app startup |

---

## 📋 Environment Variables

| Variable | Default | Description |
|----------|---------|-------------|
| `FH_SAAS_LOG_LEVEL` | WARNING | DEBUG, INFO, WARNING, ERROR |
| `FH_SAAS_LOG_FILE` | (none) | Path to log file (optional) |

## ⚙️ configure_logging

In [ ]:
#| export
def configure_logging(
    log_file: str = None,
    level: str = None,
    max_bytes: int = 10_000_000,
    backup_count: int = 5,
):
    """Configure logging for all fh_saas modules - call once at app startup."""
    # Resolve level from env var or parameter
    if level is None:
        level = os.getenv('FH_SAAS_LOG_LEVEL', 'WARNING')
    log_level = getattr(logging, level.upper(), logging.WARNING)
    
    # Resolve log file from env var or parameter
    if log_file is None:
        log_file = os.getenv('FH_SAAS_LOG_FILE')
    
    # Create formatter
    formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
    # Create handlers
    handlers = []
    handlers.append(logging.StreamHandler())  # Always console
    
    # File handler (if specified)
    if log_file:
        Path(log_file).parent.mkdir(parents=True, exist_ok=True)
        handlers.append(logging.handlers.RotatingFileHandler(
            log_file, maxBytes=max_bytes, backupCount=backup_count
        ))
    
    # Apply formatter to all handlers
    for handler in handlers:
        handler.setFormatter(formatter)
    
    # Configure fh_saas package logger
    package_logger = logging.getLogger('fh_saas')
    package_logger.setLevel(log_level)
    package_logger.handlers = handlers
    package_logger.propagate = False

In [ ]:
from nbdev.showdoc import show_doc

In [ ]:
show_doc(configure_logging)

---

### configure_logging

>      configure_logging (log_file:str=None, level:str=None,
>                         max_bytes:int=10000000, backup_count:int=5)

*Configure logging for all fh_saas modules - call once at app startup.*

## 📖 Usage Examples

### 1. App Startup (call once in main.py)

```python
from fh_saas.utils_log import configure_logging

# Option A: Use environment variables (recommended for production)
# Set FH_SAAS_LOG_LEVEL=INFO and FH_SAAS_LOG_FILE=./logs/app.log in .env
configure_logging()

# Option B: Explicit - console only
configure_logging(level='INFO')

# Option C: Console + rotating file
configure_logging(level='INFO', log_file='./logs/app.log')
```

### 2. In Any fh_saas Module (already done)

Every module in fh_saas declares a logger at the top:

```python
# fh_saas/utils_oauth.py
import logging
logger = logging.getLogger(__name__)

def handle_login_request(request, session):
    logger.debug('Login request initiated')
    # ... do work ...
    logger.info(f'OAuth complete, redirecting to {redirect_url}')
```

### 3. Where Do Logs Go?

| Configuration | Console | File |
|--------------|---------|------|
| `configure_logging()` | ✅ | ❌ |
| `configure_logging(log_file='app.log')` | ✅ | ✅ |
| No `configure_logging()` call | ❌ silent | ❌ |

### 4. Log Levels

| Level | When to Use | Example |
|-------|-------------|---------|
| `DEBUG` | Detailed diagnostic | `logger.debug('Checking user session')` |
| `INFO` | Normal operations | `logger.info('User logged in')` |
| `WARNING` | Unexpected but not failing | `logger.warning('Token expiring soon')` |
| `ERROR` | Operation failed | `logger.error('Auth failed', exc_info=True)` |

### 5. Real Example from fh_saas

```python
# In your FastHTML app
from fasthtml.common import *
from fh_saas.utils_log import configure_logging
from fh_saas.utils_oauth import handle_login_request, handle_oauth_callback

# Configure logging ONCE at startup
configure_logging(level='INFO', log_file='./logs/app.log')

app = FastHTML()

@app.get('/login')
def login(request, session):
    return handle_login_request(request, session)  # Logs automatically

@app.get('/auth/callback') 
def callback(code: str, state: str, request, session):
    return handle_oauth_callback(code, state, request, session)  # Logs automatically
```

Output in console and file:
```
2026-01-10 14:30:00 | fh_saas.utils_oauth | DEBUG | Login request initiated
2026-01-10 14:30:05 | fh_saas.utils_oauth | DEBUG | OAuth callback received
2026-01-10 14:30:05 | fh_saas.utils_oauth | INFO | OAuth complete, redirecting to /dashboard
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()